In [ ]:
%matplotlib notebook

from matplotlib import pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, confusion_matrix

### Loading the dataset

In [ ]:
dataset = pd.read_csv('../input/201710-CAH_PulseOfTheNation.csv')
dataset.head()

### Curating the data

We need to convert all the data columns of interest into discrete numerical classes. For continuous variables, this means binning them in a histogram. We will classify the answers based on:

* Gender
* Age Range
* Political Affiliation
* Race
* Education
* Income Class


In [ ]:
# First, the classes that are already discrete. Let's check the existing values with .unique()

unique_vals = {}
columns = {'gender': 'Gender', 
           'agerng': 'Age Range', 
           'polit': 'Political Affiliation ', 
           'race': 'What is your race?',
          'educ': 'What is your highest level of education?'}

for col, colname in columns.items():
    unique_vals[col] = dataset[colname].unique()
    print("{0}: {1}".format(colname, unique_vals[col]))

In [ ]:
# Now let's create a numerical index for each of them
for col, colname in columns.items():
    dataset[col + '_num'] = np.array(list(map(lambda x: list(unique_vals[col]).index(x), dataset[colname].values)))

In [ ]:
# For Income Class we need something different...

incomes = np.nan_to_num(dataset['Income'].values) # Fixes NaNs as zeroes

fig = plt.figure()
ax = fig.add_subplot(111)

ax.hist(incomes[np.where(incomes > 0)], bins=10) # Ten bins, plus one for the "zeroes" (NaNs)

In [ ]:
# We 'digitize' the class

non_zero = np.where(incomes > 0)
income_class = np.zeros(incomes.shape)

_, bins = np.histogram(incomes[non_zero])
income_class[non_zero] = np.digitize(incomes[non_zero], bins)

dataset['income_num'] = income_class.astype(int)

In [ ]:
# Now let's consider the classes we want to use as input

Xcols = [c for c in dataset.columns if '_num' in c]
dataset[Xcols].head()

### Training the model

We will try to use our inputs as predictors for one of the given questions: **"Who would you prefer as president of the United States, Darth Vader or Donald Trump?"**

In [ ]:
question = 'Who would you prefer as president of the United States, Darth Vader or Donald Trump?'

print('Given answers: ', dataset[question].unique())

dataset['ans'] = list(map(lambda x: x == 'Donald Trump', dataset[question].values))

Here we split into training and testing sets.

In [ ]:
train_set, test_set = train_test_split(dataset)

We look for the optimal value of $\alpha$, a hyperparameter in the Multinomial Naive Bayes classifier. For more information, check [the docs](http://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes).

In [ ]:
# The model, and a scan on its parameter alpha
model = MultinomialNB()
params = {
    'alpha': np.linspace(1e-10, 2.0, 20)
}

gsearch = GridSearchCV(model, params, scoring='accuracy')
gsearch.fit(train_set[Xcols], train_set['ans'])

print("Best value for alpha: {0}".format(gsearch.best_params_['alpha']))

Finally, we cross-validate the model.

In [ ]:
model = MultinomialNB(**gsearch.best_params_)
cv_scores = cross_val_score(model, X=train_set[Xcols], y=train_set['ans'], cv=5, scoring='accuracy')
print("Average CV score: {0}".format(np.average(cv_scores)))

And here we calculate a PR (Precision-Recall) curve.

In [ ]:
model.fit(train_set[Xcols], train_set['ans'])
pr, rec, thr = precision_recall_curve(test_set['ans'], model.predict_proba(test_set[Xcols])[:,1])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.set_title("PR curve")
ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.axhline(sum(test_set['ans'])*1.0/len(test_set), c=(0,0,0), ls='--', lw=0.5)
ax.plot(rec, pr)